In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=1))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [3]:
preprocessor = mz.models.aNMM.get_default_preprocessor()

In [4]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9296.68it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5986.25it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3460674.24it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10830.85it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4180.52it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 531842.50it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 6614.60it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<0

In [5]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fa1cca00e90>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fa1040e0a10>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [6]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [8]:
padding_callback = mz.models.aNMM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    shuffle=True,
    callback=padding_callback,
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    shuffle=False,
    callback=padding_callback,
)

In [9]:
model = mz.models.aNMM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix

model.params['dropout_rate'] = 0.1

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

aNMM(
  (embedding): Embedding(30058, 300)
  (matching): Matching()
  (hidden_layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (q_attention): Linear(in_features=300, out_features=1, bias=False)
  (dropout): Dropout(p=0.1)
  (out): Linear(in_features=1, out_features=1, bias=True)
) 9037903


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=15
)

trainer.run()

[Iter-102 Loss-0.689]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4755 - normalized_discounted_cumulative_gain@5(0.0): 0.5304 - mean_average_precision(0.0): 0.4953



[Iter-204 Loss-0.664]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4697 - normalized_discounted_cumulative_gain@5(0.0): 0.5263 - mean_average_precision(0.0): 0.4927



[Iter-306 Loss-0.575]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4958 - normalized_discounted_cumulative_gain@5(0.0): 0.5557 - mean_average_precision(0.0): 0.5199



[Iter-408 Loss-0.418]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5403 - normalized_discounted_cumulative_gain@5(0.0): 0.6118 - mean_average_precision(0.0): 0.5632



[Iter-510 Loss-0.248]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.582 - normalized_discounted_cumulative_gain@5(0.0): 0.6425 - mean_average_precision(0.0): 0.5923



...

[Iter-1122 Loss-0.018]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5799 - normalized_discounted_cumulative_gain@5(0.0): 0.6406 - mean_average_precision(0.0): 0.5965



[Iter-1224 Loss-0.017]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.566 - normalized_discounted_cumulative_gain@5(0.0): 0.6195 - mean_average_precision(0.0): 0.5762



[Iter-1326 Loss-0.011]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5715 - normalized_discounted_cumulative_gain@5(0.0): 0.6304 - mean_average_precision(0.0): 0.5906



[Iter-1428 Loss-0.010]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5559 - normalized_discounted_cumulative_gain@5(0.0): 0.6198 - mean_average_precision(0.0): 0.5732



[Iter-1530 Loss-0.008]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.557 - normalized_discounted_cumulative_gain@5(0.0): 0.6188 - mean_average_precision(0.0): 0.5757

Cost time: 1121.3299095630646s
